In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
train=pd.read_csv('training.csv')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Displaying the 5 rows of the dataset

In [2]:
train.head()

,id,name,document_text,cat_name
0,22474,Information Regarding the Merger of Navios Mar...,"At a special meeting held on March 24, 2021 sh...",Corporate Communications
1,27460,Announcement on Approving the Change of Member...,"On April 2, 2021, the China Financial Futures ...",Securities Settlement
2,6926,SFC Suspends Shiu Yau Wah for Five Months,The Securities and Futures Commission (SFC) ha...,Antitrust
3,6982,"Renminbi RMB Haircut - February 4, 2020",Pursuant to Section 2.6.2 of the Clearing Hous...,Securities Settlement
4,5022,"Anti-Money Laundering, Countering Financing of...",Money laundering and terrorism financing (ML/T...,Financial Crime


Displaying unique size of train attributes

In [3]:
len(np.unique(list(train['document_text'])))

9151

In [4]:
len(np.unique(list(train['id'])))

9859

In [5]:
len(np.unique(list(train['name'])))

8594

In [6]:
len(np.unique(list((train['cat_name']))))

50

In [7]:
newresult = train.groupby(['id','name'])['document_text'].agg(list).to_dict()
# for k,v in newresult:
#     print(k,v)


Cat_name groupped into one array

In [8]:
train2=train.groupby('id', sort=False).agg(lambda x: list(set(x))).reset_index()
train2.head()

train2 = train2.explode('name')


train2 = train2.explode('document_text')

train2.head()

,id,name,document_text,cat_name
0,22474,Information Regarding the Merger of Navios Mar...,"At a special meeting held on March 24, 2021 sh...",[Corporate Communications]
1,27460,Announcement on Approving the Change of Member...,"On April 2, 2021, the China Financial Futures ...","[Trade Settlement, Securities Settlement]"
2,6926,SFC Suspends Shiu Yau Wah for Five Months,The Securities and Futures Commission (SFC) ha...,"[Compliance Management, Licensure and certific..."
3,6982,"Renminbi RMB Haircut - February 4, 2020",Pursuant to Section 2.6.2 of the Clearing Hous...,"[Payments and Settlements, Securities Settleme..."
4,5022,"Anti-Money Laundering, Countering Financing of...",Money laundering and terrorism financing (ML/T...,"[Regulatory Reporting, Money-Laundering and Te..."


In [9]:
# for k,v in newresult.items():
#     print(k,v)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# initialize the TfidfVectorizer without any parameters
tfidf_vect = TfidfVectorizer() 

# with stop words removed
tfidf_vect = TfidfVectorizer(stop_words="english") 

# generate tfidf matrix
dtm= tfidf_vect.fit_transform(train["document_text"])

print("type of dtm:", type(dtm))
print("size of tfidf matrix:", dtm.shape)

type of dtm: <class 'scipy.sparse.csr.csr_matrix'>
size of tfidf matrix: (47102, 33459)


In [11]:
# 1. Check vocabulary

# Vocabulary is a dictionary mapping a word to an index

# the number of words in the vocabulary
print("total number of words:", len(tfidf_vect.vocabulary_))

print("type of vocabulary:", \
      type(tfidf_vect.vocabulary_))

total number of words: 33459
type of vocabulary: <class 'dict'>


Displaying some of the text and most freq words

In [12]:
# 3.4 check words with top tf-idf wights in a document, 
# e.g. 1st document

# get mapping from word index to word
# i.e. reversal mapping of tfidf_vect.vocabulary_
voc_lookup={tfidf_vect.vocabulary_[word]:word \
            for word in tfidf_vect.vocabulary_}

print("\nOriginal text: \n"+train["document_text"][0])

print("\ntfidf weights: \n")

# first, covert the sparse matrix row to a dense array
doc0=dtm[0].toarray()[0]
print("Vectorized document shape: ", doc0.shape, "\n")

# get index of top 20 words
print("top words:")
top_words=(doc0.argsort())[::-1][0:20]
for i in top_words:
    print("{0}:\t{1:.3f}".format(voc_lookup[i], doc0[i]))
#[(voc_lookup[i], '%.3f'%doc0[i]) for i in top_words]



Original text: 
At a special meeting held on March 24, 2021 shareholders of Navios Maritime Containers L.P. (NMCI) approved the proposed merger with Navios Maritime Partners L.P. The merger is anticipated to become effective on March 31, 2021. The details are as follows: Company Issue: Navios Maritime Containers L.P. Common Units CUSIP#: Y62151108 Symbol: NMCI Anticipated Last Trading Date: March 31, 2021 Anticipated Marketplace Effective Date for Suspension: April 1, 2021 Merger Consideration: 0.39 of a common unit representing limited partner interests in Navios Partners for each share held.

tfidf weights: 

Vectorized document shape:  (33459,) 

top words:
navios:	0.593
maritime:	0.389
nmci:	0.296
containers:	0.271
anticipated:	0.240
merger:	0.200
y62151108:	0.165
partners:	0.160
2021:	0.155
march:	0.140
held:	0.110
common:	0.108
31:	0.095
partner:	0.093
39:	0.083
representing:	0.081
date:	0.081
effective:	0.080
units:	0.072
cusip:	0.072


Using Multilabel binarizer to convert array of names into array of class variables

In [13]:
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

mlb = MultiLabelBinarizer()
# labels=list(np.unique(train2['cat_name']))
# labels=[list(labels)]
# labels
# mlb.fit(labels)
Y=mlb.fit_transform(train2['cat_name'])
# check size of indicator matrix
# print some rows 
print(Y[0:5])
print(Y.shape)
# check classes
print(mlb.classes_)

# check # of samples in each class
np.sum(Y, axis=0)

[[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 1 0 1]
 [0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
  0 0 0 0 0 0 0 1 0 0 0 1 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
  0 0 1 1 0 0 1 0 0 0 0 0 0 0]]
(9859, 50)
['Accounting and Finance' 'Antitrust' 'Banking' 'Broker Dealer'
 'Commodities Trading' 'Compliance Management' 'Consumer protection'
 'Contract Provisions' 'Corporate Communications' 'Corporate Governance'
 'Definitions' 'Delivery' 'Examinations' 'Exemptions' 'Fees and Charges'
 'Financial Accounting' 'Financial Crime' 'Forms' 'Fraud' 'IT Risk'
 'Information Filing' 'Insurance' 'Legal' 'Legal Proceedings' 'Licensing'
 'Licensure and certification' 'Liquidity 

array([ 935,  880, 1078,  670,  682, 1391,  969, 1153,  518,  958,  570,
        821, 1742, 1190, 1301,  535, 1178,  508,  906,  435, 1387,  737,
        907, 1343,  999,  982,  534, 1124,  722, 1633,  802,  869,  505,
       1079, 1099,  797,  611,  630, 1621, 1042,  627,  554,  982, 1141,
       1107,  664, 1737,  852,  872,  723])

In [28]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

# split dataset into train (70%) and test sets (30%)
X_train, X_test, Y_train, Y_test = train_test_split(\
                train2['document_text'], Y, test_size=0.30, random_state=0)

classifier = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words="english",\
                              min_df=5)),
    ('clf', OneVsRestClassifier(LinearSVC(C=3.0)))])

classifier.fit(X_train, Y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(min_df=5, stop_words='english')),
                ('clf', OneVsRestClassifier(estimator=LinearSVC(C=3.0)))])

Classification using SVM

In [29]:
from sklearn.metrics import classification_report

predicted = classifier.predict(X_test)

print(predicted.shape)
print("predicted:")
print(predicted[0:2])
print("actual:")
print(Y_test[0:2])

print(classification_report\
      (Y_test, predicted, target_names=mlb.classes_))

(2958, 50)
predicted:
[[1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
actual:
[[1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
                                          precision    recall  f1-score   support

                  Accounting and Finance       0.86      0.57      0.69       294
                               Antitrust       0.87      0.70      0.78       265
                                 Banking       0.93      0.77      0.84       344
                           Broker Dealer       0.95      0.71      0.81       220
                     Commodities Trading       0.93      0.75      0.83       195
                   Compliance Management       0.88    

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB

# split dataset into train (70%) and test sets (10%)
X_train, X_test, Y_train, Y_test = train_test_split(\
                train2['document_text'], Y, test_size=0.30, random_state=0)

classifier = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words="english",\
                              min_df=5)),
    ('clf', OneVsRestClassifier(MultinomialNB()))])

classifier.fit(X_train, Y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(min_df=5, stop_words='english')),
                ('clf', OneVsRestClassifier(estimator=MultinomialNB()))])

Classification using MultinomialNB

In [31]:
from sklearn.metrics import classification_report

predicted = classifier.predict(X_test)

print(predicted.shape)
print("predicted:")
print(predicted[0:2])
print("actual:")
print(Y_test[0:2])

print(classification_report\
      (Y_test, predicted, target_names=mlb.classes_))

(2958, 50)
predicted:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
actual:
[[1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
                                          precision    recall  f1-score   support

                  Accounting and Finance       0.86      0.02      0.04       294
                               Antitrust       0.87      0.37      0.52       265
                                 Banking       0.91      0.18      0.30       344
                           Broker Dealer       0.92      0.05      0.10       220
                     Commodities Trading       0.96      0.34      0.51       195
                   Compliance Management       0.97    

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Exercise 3.3.1 Grid search

# import pipeline class
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
# import GridSearch
from sklearn.model_selection import GridSearchCV

# build a pipeline which does two steps all together:
# (1) generate tfidf, and (2) train classifier
# each step is named, i.e. "tfidf", "clf"
C=[0.5,1,5,10,50,100,1000]
for i in C:
    print('C=',i)
    text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                         ('clf', OneVsRestClassifier(LinearSVC(C=i)))
                       ])

    # set the range of parameters to be tuned
    # each parameter is defined as 
    # <step name>__<parameter name in step>
    # e.g. min_df is a parameter of TfidfVectorizer()
    # "tfidf" is the name for TfidfVectorizer()
    # therefore, 'tfidf__min_df' is the parameter in grid search

    parameters = {'tfidf__min_df':[1,2,4,3,5,10,6,7,8,9,15,20],
                  'tfidf__stop_words':[None,"english"]
    #            'clf__C': [0.5,1.0,2.0,3,4,5,6,7,8,9,10],
    }

    # the metric used to select the best parameters
    metric =  "f1_macro"

    # GridSearch also uses cross validation
    gs_clf = GridSearchCV\
    (text_clf, param_grid=parameters, \
     scoring=metric, cv=10)

    # due to data volume and large parameter combinations
    # it may take long time to search for optimal parameter combination
    # you can use a subset of data to test
    gs_clf = gs_clf.fit(X_train, Y_train)
    for param_name in gs_clf.best_params_:
        print("{0}:\t{1}".format(param_name,\
                                     gs_clf.best_params_[param_name]))

    print("best f1 score: {:.3f}".format(gs_clf.best_score_))


C= 0.5

tfidf__min_df:	1

tfidf__stop_words:	english

best f1 score: 0.710

C= 1


In [57]:
for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name,\
                                 gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))


tfidf__min_df:	20

tfidf__stop_words:	english

best f1 score: 0.450
